#### Load Modules

In [2]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch
import random 

#Import my modules
import run_expts
import format_eeg_data
import constants
import eeg_stat_ts

#Import classifiers 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


#### Load Data

In [23]:
#Load Wake Data + Sleep data combined
y = joblib.load('y_concat.pkl')
groups = joblib.load('groups_concat.pkl')
X = joblib.load('concat_df.pkl')

In [24]:
#Load Sleep data combined
y = joblib.load('y_sleep_concat.pkl')
groups = joblib.load('groups_sleep_concat.pkl')
X = joblib.load('sleep_concat_df.pkl')

In [4]:
y.value_counts()

y_REM
0    25
3    22
1    14
2     6
Name: count, dtype: int64

#### Run expts using X , y , groups

In [5]:
clf_dict = {'RF' : RandomForestClassifier(random_state = 5) , 'SVC' : SVC(), 'Ada_B' : AdaBoostClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier()}

In [6]:
#Generate a region to features dictionary - this will enable us to run expts regionally as before
regional_features_dict = {}
region_channel_dict = constants.region_to_channel_dict
regions = list(region_channel_dict.keys())

sum = 0

for region in regions: 
    region_cols = [col for col in X.columns if '_' + region + '_' in col]
    regional_features_dict[region] = region_cols
    sum += len(region_cols)

sum

572

##### Custom Region - Features dictionary. Randomly reduce features

In [65]:
# Generate a region to features dictionary
# This dictionary will trim the number of features from each data type randomly to see if this helps classification

data_types = ['N1' , 'N2', 'N3','REM']
data_types = ['N1'] 

regional_features_dict = {}
region_channel_dict = constants.region_to_channel_dict
regions = list(region_channel_dict.keys())

sum = 0

for region in regions: 
    region_cols = [col for col in X.columns if '_' + region + '_' in col]
    selected_cols = []
    for data_type in data_types:
        data_type_cols = [col for col in region_cols if data_type in col]
        selected_cols = data_type_cols
        # random_selection = random.sample(data_type_cols, 8)
        # selected_cols += random_selection
    
    regional_features_dict[region] = selected_cols
    sum += len(region_cols)


##### Run the experiments

In [66]:
for key in regional_features_dict.keys():
    print(len(regional_features_dict[key]) )

11
11
11
11
11
11
11
11
11
11
11
11
11


In [54]:
y_expt.value_counts()

y_REM
0    25
1    20
Name: count, dtype: int64

In [67]:
t1 = time.time()

#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, clf_dict , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )

t2 = time.time()

t2 - t1

19.816067695617676

In [68]:
results_df

,random_state,y_true,Prefrontal_RF_y_preds,Prefrontal_SVC_y_preds,Prefrontal_Ada_B_y_preds,Prefrontal_DT_y_preds,Frontal_RF_y_preds,Frontal_SVC_y_preds,Frontal_Ada_B_y_preds,Frontal_DT_y_preds,...,Left Parietal_Ada_B_y_preds,Left Parietal_DT_y_preds,Right Parietal_RF_y_preds,Right Parietal_SVC_y_preds,Right Parietal_Ada_B_y_preds,Right Parietal_DT_y_preds,Occipital_RF_y_preds,Occipital_SVC_y_preds,Occipital_Ada_B_y_preds,Occipital_DT_y_preds
0,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, ...","[[1, 1, 0, 1, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, ...","[[1, 1, 0, 1, 1, 0, 0, 0, 1], [1, 0, 1, 0, 0, ...","[[1, 1, 0, 1, 1, 0, 0, 0, 1], [1, 1, 1, 0, 0, ...","[[1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 0, 0, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 1, 0, 1, 1], [1, 1, 1, 1, 0, ...","[[1, 1, 0, 1, 1, 1, 0, 1, 1], [0, 0, 0, 0, 0, ...",...,"[[0, 1, 1, 1, 0, 1, 1, 0, 0], [1, 1, 1, 0, 0, ...","[[0, 1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 0, 1, ...","[[0, 1, 1, 1, 1, 1, 0, 1, 1], [0, 1, 1, 0, 1, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, ...","[[1, 0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, ...","[[1, 1, 1, 1, 0, 0, 1, 0, 1], [1, 1, 1, 0, 0, ...","[[1, 0, 1, 1, 1, 0, 0, 1, 1], [1, 1, 1, 0, 0, ..."
1,2,"[[0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, ...","[[1, 0, 1, 0, 0, 0, 1, 1, 1], [1, 1, 0, 0, 1, ...","[[0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 0, ...","[[0, 0, 1, 1, 0, 0, 1, 0, 1], [0, 1, 0, 1, 1, ...","[[0, 0, 1, 1, 0, 1, 1, 1, 1], [0, 1, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 0, 1, 1, 1], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 0, 1, 1, 1], [0, 0, 0, 1, 0, ...","[[1, 1, 0, 0, 1, 0, 1, 1, 1], [0, 0, 0, 0, 0, ...",...,"[[1, 1, 1, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, ...","[[0, 1, 1, 0, 0, 1, 0, 0, 1], [0, 1, 1, 1, 1, ...","[[0, 1, 1, 0, 0, 1, 1, 0, 1], [1, 1, 1, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 0, 0, ...","[[0, 1, 0, 0, 0, 1, 0, 0, 1], [1, 1, 1, 1, 1, ...","[[1, 1, 1, 1, 0, 1, 1, 0, 1], [1, 1, 1, 0, 0, ...","[[1, 1, 1, 0, 0, 1, 0, 0, 1], [1, 0, 1, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 0, 0, 0, ...","[[1, 1, 1, 0, 0, 1, 0, 0, 1], [1, 0, 1, 1, 1, ...","[[1, 0, 1, 0, 0, 1, 0, 1, 1], [1, 1, 0, 1, 1, ..."


In [69]:
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

In [70]:
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T )
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'SVC' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)

,Prefrontal_DT_acc,Frontal_DT_acc,Left Frontal_DT_acc,Right Frontal_DT_acc,Central_DT_acc,Left Central_DT_acc,Right Central_DT_acc,Left Temporal_DT_acc,Right Temporal_DT_acc,Parietal_DT_acc,Left Parietal_DT_acc,Right Parietal_DT_acc,Occipital_DT_acc
0,0.533333,0.444444,0.355556,0.455556,0.444444,0.466667,0.466667,0.277778,0.4,0.444444,0.455556,0.455556,0.488889


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.522222,0.4,0.366667,0.422222,0.355556,0.366667,0.477778,0.255556,0.411111,0.4,0.5,0.366667,0.411111


,Prefrontal_SVC_acc,Frontal_SVC_acc,Left Frontal_SVC_acc,Right Frontal_SVC_acc,Central_SVC_acc,Left Central_SVC_acc,Right Central_SVC_acc,Left Temporal_SVC_acc,Right Temporal_SVC_acc,Parietal_SVC_acc,Left Parietal_SVC_acc,Right Parietal_SVC_acc,Occipital_SVC_acc
0,0.477778,0.411111,0.4,0.433333,0.4,0.422222,0.422222,0.433333,0.422222,0.466667,0.466667,0.455556,0.455556


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.477778,0.444444,0.288889,0.344444,0.444444,0.5,0.466667,0.333333,0.377778,0.422222,0.433333,0.377778,0.4
